In [2]:
%matplotlib osx
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits import mplot3d

# Fetching the simulation parameters and the actual data

In [3]:
# default values
Nmax = 128
tdiv = 500
n_genes = 5

sim_id = "Fig1c_2023_01_03-12_55"
# fetching the simulation parameters
dir = "simulation_output/" + sim_id
file = open(dir + "/abstract.txt","r")
for line in file:
    line = line.replace(" ","")
    line = line.replace("\n","")
    line = line.split("=")

    if(line[0] == "Nmax"):
        Nmax = int(line[1])
    elif(line[0] == "theta"):
        n_genes = len(line[1].split(","))
    elif(line[0] == "tdiv"):
        tdiv = int(line[1])                 # ill-named: This is the number of updates to the expresion levels that are carried out, not
                                            # the time in seconds between cell divisions
    elif(line[0] == "dt"):
        dt = float(line[1])
file.close()

# fetching the simulation data
p = []
m = []
for i in range(Nmax):
    # reading in every protein and mRNA timeline for every cell
    p += [np.loadtxt(dir + "/p_cell{}.txt".format(i),delimiter=","),]
    m += [np.loadtxt(dir + "/m_cell{}.txt".format(i),delimiter=","),]
p = np.array(p)
m = np.array(m)

print("Nmax = {}\ntdiv = {}\nn_genes = {}\ndt = {}".format(Nmax,tdiv,n_genes,dt))
print("p.shape = ",p.shape)
print("m.shape = ",m.shape)

Nmax = 128
tdiv = 5000
n_genes = 5
dt = 0.01
p.shape =  (128, 40001, 5)
m.shape =  (128, 40001, 5)


# Creating a 3D-plot of the mRNA concentration

In [4]:
# start and stop indices for the time interval over which we are plotting (times in seconds)
t_start = 50
t_stop = 100
i_start = t_start / dt
i_stop = t_stop / dt

if int(i_start) != i_start or int(i_stop) != i_stop:
    print("int(i_start) != i_start")
if int(i_stop) != i_stop:
    print("int(i_stop) != i_stop")
if(int(i_start) == i_start and int(i_stop) == i_stop):
    i_start = int(i_start)
    i_stop = int(i_stop)

fig = plt.figure("Time Evolution in protein space for all cells and all times")
ax = plt.axes(projection='3d')
color = iter(plt.cm.jet(np.linspace(start=0,stop=1,num=Nmax)))
for i in range(Nmax):
    ax.plot3D(p[i,i_start:i_stop,2],p[i,i_start:i_stop,3],p[i,i_start:i_stop,4],c=next(color),label="cell {}".format(i))
ax.set_xlabel("p[2] / 1")
ax.set_ylabel("p[3] / 1")
ax.set_zlabel("p[4] / 1")
# plt.legend()
plt.show()

# Crearing time series plots of the protein concentration
I will create five plots; one for each protein. I will plot the protein concentration over time, including new cells after each division.

In [5]:
max_n_cells = 128
if int(np.log2(max_n_cells)) != np.log2(max_n_cells):       # Sanity Check
    print("Non-integer result for np.log2(max_n_cells) !")
time_axis = np.linspace(start=1,stop=tdiv * (np.log2(max_n_cells)+1),num=tdiv * int((np.log2(max_n_cells)+1)))*dt

fi,ax = plt.subplots(nrows=5,ncols=1,sharex=True,num="Protein concentration for up to {} cells".format(max_n_cells),figsize=(30,10))
for i in range(5):                                  # Loop over the proteines
    for j in range(int(np.log2(max_n_cells))+1):        # loop over times
        ax[i].axvline(tdiv*j*dt,c="black")
    color = iter(plt.cm.jet(np.linspace(start=0,stop=1,num=Nmax)))
    for k in range(Nmax):                               # loop over all cells
        ax[i].plot(time_axis,p[k,0:int(np.log2(max_n_cells)+1)*tdiv,i],c=next(color))
    ax[i].set_ylabel("p[{}] / 1".format(i))
    ax[i].grid()
ax[4].set_xlabel(r"time $t \:/\: s$")

# creating a second axis which shows the cell number
ax_n = ax[0].twiny()
new_tick_locations = [tdiv*(j+1/2)*dt for j in range(int(np.log2(max_n_cells))+1)]
new_tick_labels = [2**j for j in range(int(np.log2(max_n_cells))+1)]
ax_n.set_xlim(ax[0].get_xlim())
ax_n.set_xticks(new_tick_locations)
ax_n.set_xticklabels(new_tick_labels)
ax_n.set_xlabel(r"Cell number $N$")

plt.savefig("plots/" + sim_id + "_history.pdf",bbox_inches="tight")

# Determining if there are different cell types
I will cluster the cells into cell types by the distance measure introduced in the paper. I will assume that "being of the same cell type" is transitive, meaning if $a$ and $b$ are the same and $b$ and $c$ are the same, then $a$ and $c$ are the same.

In [6]:
# start and stop indices for the time interval over which we are measuring
N_cells_measure = 128
t_start = 0             # within the time span when there were N_cells_measure cells, in seconds
delta_t = 50
i_start = np.log2(N_cells_measure)*tdiv + t_start / dt
i_stop = i_start + delta_t / dt

if int(i_start) != i_start or int(i_stop) != i_stop:
    print("int(i_start) != i_start")
if int(i_stop) != i_stop:
    print("int(i_stop) != i_stop")
if(int(i_start) == i_start and int(i_stop) == i_stop):
    i_start = int(i_start)
    i_stop = int(i_stop)

# computing the average protein expression levels
p_shape = (Nmax,n_genes)
p_average = np.zeros(shape=p_shape)

for i in range(p.shape[0]):
    for k in range(n_genes):
        p_average[i,k] = np.mean(p[i,i_start:i_stop,k])

# computing the distances
dist = np.zeros(shape=(Nmax,Nmax))
for i in range(Nmax):
    for j in range(Nmax):
        dist[i,j] = np.sqrt(np.sum((p_average[i,:] - p_average[j,:])**2))

# the array cell_types will contain an array for each cell type. cell_types[i] will contain all cells of the respective type
cell_types = []
threshold = .3
# clustering
for i in range(Nmax):
    matched = False
    # searching for a cell of matching type in the known cell types
    for cell_type in cell_types:
        for j in cell_type:
            if dist[i,j] < threshold:
                # we found a match in one of the existing cell types
                cell_type += [i,]
                matched = True
                break
        if matched: break
    
    if not matched:
        # we did not find a match; the current cell has a new type
        cell_types += [[i],]

# sanity check
sum = 0
for cell_type in cell_types: sum += len(cell_type)
if(sum != Nmax):
    print("You lost some cells on the way")

print("Between t = {} and t = {} in the {}-cell stage of the simulation, there were {} distinct cell types.".format(t_start,t_start+delta_t,N_cells_measure,len(cell_types)))

Between t = 0 and t = 50 in the 128-cell stage of the simulation, there were 2 distinct cell types.


# For sim_id = "Fig1c_2023_01_03-12_55": Creating a plot that shows how a differentiated cell type is an attractor in the protein space

In [7]:
if False:
    max_n_cells = 128
    if int(np.log2(max_n_cells)) != np.log2(max_n_cells):       # Sanity Check
        print("Non-integer result for np.log2(max_n_cells) !")
    time_axis = np.linspace(start=1,stop=tdiv * (np.log2(max_n_cells)+1),num=tdiv * int((np.log2(max_n_cells)+1)))*dt

    fig = plt.figure("Protein 2 expression level for up to {} cells".format(max_n_cells),figsize=(30,10))
    ax = plt.gca()
    for j in range(int(np.log2(max_n_cells))+1):        # loop over times
        ax.axvline(tdiv*j*dt,c="black")
    color = iter(plt.cm.jet(np.linspace(start=0,stop=1,num=Nmax)))
    for k in range(Nmax):                               # loop over all cells
        ax.plot(time_axis,p[k,0:int(np.log2(max_n_cells)+1)*tdiv,2],c=next(color))
    ax.set_ylabel("p[{}] / 1".format(2))
    ax.grid()
    ax.set_xlabel(r"time $t \:/\: s$")

    # creating a second axis which shows the cell number
    ax_n = ax.twiny()
    new_tick_locations = [tdiv*(j+1/2)*dt for j in range(int(np.log2(max_n_cells))+1)]
    new_tick_labels = [2**j for j in range(int(np.log2(max_n_cells))+1)]
    ax_n.set_xlim(ax.get_xlim())
    ax_n.set_xticks(new_tick_locations)
    ax_n.set_xticklabels(new_tick_labels)
    ax_n.set_xlabel(r"Cell number $N$")

    # plt.savefig("plots/" + sim_id + "_history.pdf",bbox_inches="tight")

Text(0.5, 0, 'Cell number $N$')

2023-01-07 01:16:51.571 python[57915:2884476] +[CATransaction synchronize] called within transaction
